# 数据清洗和准备

主要讨论处理缺失数据、重复数据、字符串操作和其他数据转换的工具。

In [134]:
import numpy as np
import pandas as pd

## 处理缺失数据

Pandas 的目标之一就是尽量轻松地处理缺失值，比如 Pandas 所有的描述性统计默认都不包括缺失数据。

![表7-1 NA处理方法](Img/7178691-1a0f73e5bb26ea21.png)

In [135]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
string_data[0] = None  # 内置对象 None 可以作为 NA
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 滤除缺失数据

- `pandas.isnull()`或布尔索引的手工方法
- `dropna`

In [6]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
# 等价于：
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
# DataFrame 删除 NA
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

# dropna 默认删除任何含有缺失的行
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [10]:
# 只删除全为 NA 的行
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [11]:
# 删除全为缺失的列
data[4] = NA
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
# 只想留下一部分观测数据
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.219293,NaN,NaN
1,0.569547,NaN,NaN
2,-0.344530,NaN,0.812745
3,0.698073,NaN,0.462002
4,-0.955175,0.023522,-0.323103
5,-1.699145,0.171593,-0.265144
6,0.939183,-1.955625,-0.479548


In [17]:
df.dropna()

,0,1,2
4,-0.955175,0.023522,-0.323103
5,-1.699145,0.171593,-0.265144
6,0.939183,-1.955625,-0.479548


In [18]:
df.dropna(thresh=2)

,0,1,2
2,-0.344530,NaN,0.812745
3,0.698073,NaN,0.462002
4,-0.955175,0.023522,-0.323103
5,-1.699145,0.171593,-0.265144
6,0.939183,-1.955625,-0.479548


### 填充缺失值

`fillna()` 是主要函数,对reindex有效的那些插值方法也可用于fillna。

`fillna`的参数说明：

![](Img/7178691-0bf235386a64c3b5.png)
![fillna函数参数](Img/7178691-4edd39e68f4dc530.png)

In [19]:
df.fillna(0)  # 替换为0

,0,1,2
0,-0.219293,0.000000,0.000000
1,0.569547,0.000000,0.000000
2,-0.344530,0.000000,0.812745
3,0.698073,0.000000,0.462002
4,-0.955175,0.023522,-0.323103
5,-1.699145,0.171593,-0.265144
6,0.939183,-1.955625,-0.479548


In [20]:
# 通过字典调用：实现对不同的列填充不同的值
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.219293,0.500000,0.000000
1,0.569547,0.500000,0.000000
2,-0.344530,0.500000,0.812745
3,0.698073,0.500000,0.462002
4,-0.955175,0.023522,-0.323103
5,-1.699145,0.171593,-0.265144
6,0.939183,-1.955625,-0.479548


In [21]:
# 就地修改
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.219293,0.000000,0.000000
1,0.569547,0.000000,0.000000
2,-0.344530,0.000000,0.812745
3,0.698073,0.000000,0.462002
4,-0.955175,0.023522,-0.323103
5,-1.699145,0.171593,-0.265144
6,0.939183,-1.955625,-0.479548


In [22]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-0.840029,0.956791,1.031808
1,-0.852067,0.386887,-0.833720
2,-1.377122,NaN,-0.346766
3,-0.672295,NaN,-0.106684
4,0.160814,NaN,NaN
5,-0.607895,NaN,NaN


In [23]:
df.fillna(method='ffill')

,0,1,2
0,-0.840029,0.956791,1.031808
1,-0.852067,0.386887,-0.833720
2,-1.377122,0.386887,-0.346766
3,-0.672295,0.386887,-0.106684
4,0.160814,0.386887,-0.106684
5,-0.607895,0.386887,-0.106684


In [24]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.840029,0.956791,1.031808
1,-0.852067,0.386887,-0.833720
2,-1.377122,0.386887,-0.346766
3,-0.672295,0.386887,-0.106684
4,0.160814,NaN,-0.106684
5,-0.607895,NaN,-0.106684


In [26]:
# 可以实现其他功能
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())  # 传入 Series 的平均值

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 数据转换

之前介绍了数据的重排，另一类重要的操作是过滤、清理以及其他的转换工作。

### 移除重复数据

In [27]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [28]:
# duplicated 方法
data.duplicated()  # 返回布尔类型 Series

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [29]:
# drop_duplicates 方法
data.drop_duplicates()  # droped : two 4

# 1.默认删除行列完全一样的
# 2.默认会判断全部列
# 3.默认保留第一次出现的值

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [32]:
# 指定判断列
data['v1'] = range(7)

In [30]:
# 指定被判断的列
data.drop_duplicates(['k1'])

,k1,k2
0,one,1
1,two,1


In [33]:
# duplicated和drop_duplicates默认保留的是第一个出现的值组合
# 传入keep='last'则保留最后一个

data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 利用函数和映射进行数据转换

使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。


In [35]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [36]:
# 添加（肉类 -> 来源）的映射
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [40]:
# map 方法：可以接受函数或者映射关系的字典型对象
lowercased = data['food'].str.lower()  # 肉类首字母小写
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [41]:
# 使用匿名函数
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

#### 替换值

利用fillna方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式。

>笔记：data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换。我们会在后面学习Series的字符串方法。

In [42]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [43]:
# 将 -999 替换为缺失
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# 一次性替换多个值
data.replace([-999, -1000], np.nan)

In [ ]:
# 替换成不同的值
data.replace([-999, -1000], [np.nan, 0])

In [44]:
# 传入字典也行
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引


In [45]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [46]:
def transform(x): return x[:4].upper()


data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [48]:
# 就地修改
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [49]:
# 创建数据集的转换版：rename 方法
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [50]:
# rename 结合字典对象实现对部分轴标签的更新
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [52]:
# 就地修改
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 离散化和面元划分
为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。

假设有一组人员数据，而你希望将它们划分为不同的年龄组：

In [54]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [56]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是一个特殊的Categorical对象。结果展示了pandas.cut划分的面元。你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：

In [57]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [58]:
cats.categories

# 1.区间和数据符号一样，圆括号表示开端，方括号表示闭端
# 2.那边闭端可以通过right=False进行修改

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [59]:
pd.value_counts(cats)  # pandas.cut结果的面元计数

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [60]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [61]:
# 传递列表或数组到labels，自定义面元名称
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [63]:
# 如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。
# 下面这个例子中，我们将一些均匀分布的数据分成四组：

data = np.random.rand(20)
pd.cut(data, 4, precision=2)  # precision：限定小数只有两位

[(0.74, 0.98], (0.74, 0.98], (0.25, 0.49], (0.49, 0.74], (0.0017, 0.25], ..., (0.49, 0.74], (0.74, 0.98], (0.74, 0.98], (0.0017, 0.25], (0.49, 0.74]]
Length: 20
Categories (4, interval[float64]): [(0.0017, 0.25] < (0.25, 0.49] < (0.49, 0.74] < (0.74, 0.98]]

In [64]:
# qcut：根据样本分位数进行面元划分
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-2.916, -0.716], (-0.716, -0.0379], (-2.916, -0.716], (-2.916, -0.716], (-0.716, -0.0379], ..., (0.623, 3.151], (-0.716, -0.0379], (0.623, 3.151], (-2.916, -0.716], (-2.916, -0.716]]
Length: 1000
Categories (4, interval[float64]): [(-2.916, -0.716] < (-0.716, -0.0379] < (-0.0379, 0.623] < (0.623, 3.151]]

In [65]:
pd.value_counts(cats)

(0.623, 3.151]       250
(-0.0379, 0.623]     250
(-0.716, -0.0379]    250
(-2.916, -0.716]     250
dtype: int64

In [66]:
# 传递自定义的分位数
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.309, -0.0379], (-1.309, -0.0379], (-1.309, -0.0379], (-2.916, -1.309], (-1.309, -0.0379], ..., (-0.0379, 1.22], (-1.309, -0.0379], (-0.0379, 1.22], (-2.916, -1.309], (-1.309, -0.0379]]
Length: 1000
Categories (4, interval[float64]): [(-2.916, -1.309] < (-1.309, -0.0379] < (-0.0379, 1.22] < (1.22, 3.151]]

### 检测和过滤异常值

过滤或变换异常值（outlier）在很大程度上就是运用数组运算。

In [68]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.019222,0.039727,-0.000533,-0.021265
std,1.001470,0.980492,1.011572,1.030032
min,-3.455901,-3.111718,-2.896119,-3.538318
25%,-0.664691,-0.580014,-0.722431,-0.681912
50%,0.045695,0.024134,-0.028180,-0.009897
75%,0.683664,0.684816,0.699560,0.642782
max,3.107005,2.714924,3.062679,3.293724


In [69]:
# 找出某列中绝对值大于3的值
col = data[2]
col[np.abs(col) > 3]

336    3.062679
Name: 2, dtype: float64

In [72]:
# 选出全部含有超过3或者-3的行
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
57,3.107005,1.711405,0.337822,-1.001630
220,-3.321979,0.391698,-0.441572,-0.459339
269,0.077593,-0.279581,-0.295580,-3.264789
290,-3.125721,0.705947,0.609459,0.313922
336,-0.314547,0.523824,3.062679,-2.427585
491,1.192393,0.285576,-0.570392,-3.538318
496,-1.500873,-3.111718,-2.110813,-0.032680
728,0.053463,-2.106490,0.342410,3.256893
737,-3.455901,-1.514222,-0.046206,-0.081215
757,-0.610159,-0.104439,-0.257740,3.293724


In [73]:
# 将计算限定在[-3,3]之间
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.020018,0.039838,-0.000596,-0.021012
std,0.998249,0.980139,1.011384,1.025834
min,-3.000000,-3.000000,-2.896119,-3.000000
25%,-0.664691,-0.580014,-0.722431,-0.681912
50%,0.045695,0.024134,-0.028180,-0.009897
75%,0.683664,0.684816,0.699560,0.642782
max,3.000000,2.714924,3.000000,3.000000


In [74]:
# sign():数据的值是正还是负，
# np.sign(data)可以生成1和-1

np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,1.0,1.0,-1.0,1.0
2,-1.0,-1.0,1.0,-1.0
3,1.0,1.0,1.0,-1.0
4,1.0,1.0,1.0,-1.0


### 排列和随机采样

利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组。

In [75]:
df = pd.DataFrame(np.arange(5*4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 4, 2, 0, 3])

In [76]:
df  # 之后可以基于 iloc 索引操作或在take函数中使用
df.take(sampler)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3
3,12,13,14,15


In [77]:
# 使用 sample 方法随机选取
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
0,0,1,2,3


In [78]:
# 允许重复选择
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

4    4
0    5
0    5
4    4
2   -1
2   -1
2   -1
2   -1
0    5
2   -1
dtype: int64

### 计算指标和哑变量

另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。

In [80]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [81]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [82]:
# 给指标的DataFrame加上前缀
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [84]:
# 如果某行同属于多个分类？
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat',
                       sep='::',
                       header=None,
                       names=mnames)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [85]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [86]:
# 如何为每个genre添加指标变量？

# 从数据集抽取不同的genre值
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

In [88]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [89]:
# 从一个全为零的DataFrame开始
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [90]:
# 迭代每一步电影
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [91]:
# 根据索引，使用 iloc 设定值
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [92]:
# 再将其与movies合并
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

>笔记：对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入NumPy数组，然后结果包装在DataFrame中。

In [93]:
# 结合get_dummies和诸如cut之类的离散化函数
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [94]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 字符串操作

- 大部分文本运算都直接做成了字符串对象的内置方法
- 更为复杂的模式匹配和文本操作，提供了正则表达式

Python内置的字符串方法:
![](Img/7178691-087fe67bf6db0701.png)
![](Img/7178691-d1f0d4ed3e895016.png)

In [ ]:
# 字符串对象方法

In [95]:
# split(): 切割
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [96]:
# strip():去除空白符
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [ ]:
# + : 字符串连接
first, second, third = pieces
first + '::' + second + '::' + third

In [97]:
# join():字符串拼接
'::'.join(pieces)

'a::b::guido'

In [98]:
# 子串定位:in, index, find
'guido' in val

True

In [99]:
val.index(',')

1

In [100]:
val.find('n,')

# 注意：find和index的区别：如果找不到字符串，
# index将会引发一个异常（而不是返回－1）

-1

In [101]:
val.index("$")

ValueError: substring not found

In [103]:
# 子串出现的次数
val.count(',')

2

In [104]:
# 替换子串
val.replace(',', '::')
val.replace(',', '')

'a::b:: guido'

In [106]:
# casefold: 字符转换为小写
val.replace(',', '').casefold()

'ab guido'

## 正则表达式

正则表达式提供了一种灵活的在文本中搜索或匹配（通常比前者复杂）字符串模式的方式。正则表达式，常称作regex，是根据正则表达式语言编写的字符串。Python内置的re模块负责对字符串应用正则表达式。

re模块的函数可以分为三个大类：**模式匹配、替换以及拆分。**

![](Img/7178691-efbb80a793759fc0.png)

In [107]:
# 需求：拆分一个字符串，
# 分隔符为数量不定的一组空白符（制表符、空格、换行符等）
import re
text = "foo bar\t baz \tqux"

re.split('\s+', text)  # \s+：一个或多个空白符

# 调用re.split('\s+',text)时，
# 正则表达式会先被编译，
# 然后再在text上调用其split方法。

['foo', 'bar', 'baz', 'qux']

In [108]:
# 使用 re.compile编译regex
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
# 只希望得到匹配regex的所有模式
regex.findall(text)

>如果打算对许多字符串应用同一条正则表达式，强烈建议通过re.compile创建regex对象。这样将可以节省大量的CPU时间。

In [109]:
# match和search跟findall功能类似。
# findall返回的是字符串中所有的匹配项，而search则只返回第一个匹配项。
# match更加严格，它只匹配字符串的首部。

text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [110]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [111]:
m = regex.search(text)
m  # 返回第一个电子邮件
# span(5:20)：在原字符串的起始和结束位置

<re.Match object; span=(5, 20), match='dave@google.com'>

In [112]:
text[m.start():m.end()]

'dave@google.com'

In [113]:
# regex.match则将返回None,
# 因为它只匹配出现的字符串开头的模式
print(regex.match(text))

None


In [114]:
# sub方法可以将匹配到的模式替换为指定字符串
# 并返回所得到的新字符串

print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [117]:
# 需求：将各个地址分成3部分：用户名、域名以及域名后缀
# 待分段的模式分组（各部分用圆括号包）
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [118]:
# 所产生的匹配对象，可以通过 groups() 方法返回一个由
# 模式各段组成的元组
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [119]:
# 带有分组模式，findall会返回一个元组列表
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [ ]:
# sub 通过 \1,\2 特殊符号访问各匹配项中的分组
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

## pandas 的矢量化字符串函数

清理待分析的散乱数据时，常常需要做一些字符串规整化工作。更为复杂的情况是，含有字符串的列有时还含有缺失数据。

![表7-5 部分矢量化字符串方法](Img/7178691-a634364ed6d5d5c5.png)

In [120]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [121]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

通过data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。为了解决这个问题，Series有一些能够跳过NA值的面向数组方法，进行字符串操作。通过Series的str属性即可访问这些方法。例如，我们可以通过str.contains检查各个电子邮件地址是否含有"gmail"：

In [ ]:
data.str.contains('gmail')

In [122]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [123]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [128]:
# 实现矢量化的元素获取：
# 1.使用 str.get；2.str属性上使用索引
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [133]:
# 访问嵌入列表中的元素
# matches.str.get(1)
# matches.str[0]

In [132]:
data.str[:5]  # 字符串截取

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object